In [1]:
# Importing of dependencies

import pandas as pd
import lxml
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import urllib.request

In [2]:
home = 'https://www.flipkart.com'
link = 'https://www.flipkart.com/laptops/pr?sid=6bo%2Cb5g'

def scraper():
    df = pd.DataFrame()
    data = requests.get(link)
    soup = BeautifulSoup(data.text,'lxml')
    
    mydiv = soup.find('div',{'class' : '_2MImiq'})
    no_pgs = mydiv.find('span').text.split()[-1]
    no_pgs = int(no_pgs)
    
    for page in tqdm(range(1,no_pgs+1)):
        url = link + (f"&page={page}")
        data = requests.get(url)
        soup = BeautifulSoup(data.text,'lxml')

        mylinks = soup.find_all('a', {'class' : '_1fQZEK'})

        id_cnt = 1

        for laplink in mylinks:
            lap_url = home + laplink['href']
            data = requests.get(lap_url)
            soup = BeautifulSoup(data.text,'lxml')

            try:
                brand = soup.find('span',{'class' : 'B_NuCI'}).text.split()[0].upper()
            except:
                continue

            table_vals = soup.find_all('tr', {'class' : '_1s_Smc'})
            
            # Inital Values
            proc_data = ""
            has_ssd = 0
            hdd = "None"
            
            for val in table_vals:
                thead = val.find('td', {'class' : '_1hKmbr'}).text
                tdata = val.find('li', {'class' : '_21lJbe'}).text

                # General Specs
                if(thead == "Type"): lap_type = tdata
                elif(thead == "Suitable For"): suit = tdata

                # Processor and Memory Specs
                elif(thead == "Processor Brand"): proc_data += (tdata + " ")
                elif(thead == "Processor Name"): proc_data += (tdata + " ")
                elif(thead == "Processor Generation"): proc_data += (tdata + " ")
                elif(thead == "SSD"): has_ssd |= (1 if tdata == 'Yes' else 0)
                elif(thead == "SSD Capacity"): ssd_cap = tdata
                elif(thead == "Graphic Processor"): gp_name = tdata
                elif(thead == "RAM"): ram = tdata
                elif(thead == "HDD Capacity"): hdd = tdata
                
                # Operating System Specs
                elif(thead == "Operating System"): os = tdata

                # Display and Audio Specs
                elif(thead == "Touchscreen"): touch = tdata            
                elif(thead == "Screen Size"): sze = tdata
                elif(thead == "Screen Resolution"): res = tdata

                # Dimension Specs
                elif(thead == "Weight"): weight = tdata

            price = soup.find('div',{'class' : '_30jeq3 _16Jk6d'}).text

            new_row = pd.DataFrame({
                'id' : id_cnt
                , 'Brand' : brand
                , 'Type' : lap_type
                , 'Suits' : suit
                , 'CPUbrand' : proc_data
                , 'HDD' : hdd
                , 'SSD' : 'None' if has_ssd == 0 else ssd_cap
                , 'GPUbrand' : gp_name
                , 'RAM' : ram
                , 'OS' : os
                , 'Touchscreen' : touch
                , 'ScreenSize' : sze
                , 'Resolution' : res
                , 'Weight' : weight
                , 'Price' : price
            }, index = [0])

            df = pd.concat([df.loc[:], new_row]).reset_index(drop = True)

            df.to_csv('Scraper-Data.csv')
            id_cnt += 1
    return df

In [3]:
ldf = scraper()

100%|██████████████████████████████████████████████████████████████████████████████████| 54/54 [09:06<00:00, 10.13s/it]


In [6]:
ldf

,id,Brand,Type,Suits,CPUbrand,SSD,GPU,RAM,OS,Touchscreen,ScreenSize,Resolution,Weight,Price
0,1,REDMIBOOK,Thin and Light Laptop,Processing & Multitasking,Intel Core i3 11th Gen,256 GB,Intel Integrated UHD,8 GB,Windows 10 Home,No,39.62 cm (15.6 inch),1920 x 1080 Pixel,1.8 kg,"₹27,990"
1,2,ASUS,Thin and Light Laptop,Processing & Multitasking,AMD Ryzen 3 Dual Core,256 GB,AMD Radeon AMD,8 GB,Windows 11 Home,No,39.62 cm (15.6 inch),1366 x 768 Pixel,1.80 kg,"₹28,990"
2,3,HP,Thin and Light Laptop,"Everyday Use, Processing & Multitasking",Intel Core i3 11th Gen,512 GB,Intel Integrated UHD Graphics,8 GB,Windows 11 Home,No,39.62 cm (15.6 Inch),1920 x 1080 Pixel,1.69 Kg,"₹40,990"
3,4,ASUS,Thin and Light Laptop,Processing & Multitasking,Intel Core i5 10th Gen,512 GB,Intel Integrated UHD,8 GB,Windows 11 Home,No,39.62 cm (15.6 inch),1920 x 1080 Pixel,1.80 kg,"₹44,990"
4,5,ACER,Gaming Laptop,Gaming,AMD Ryzen 5 Hexa Core,512 GB,NVIDIA GeForce GTX 1650,8 GB,Windows 11 Home,No,39.62 cm (15.6 inch),1920 x 1080 Pixel,2.15 kg,"₹45,990"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,20,HP,Thin and Light Laptop,Processing & Multitasking,AMD Ryzen 5 Hexa Core,512 GB,AMD Radeon Radeon Graphics,16 GB,Windows 11 Home,No,39.62 cm (15.6 Inch),1920 x 1080 Pixels,1.75 Kg,"₹67,500"
980,21,HP,Thin and Light Laptop,Processing & Multitasking,AMD Ryzen 5 Hexa Core,512 GB,Intel Integrated UHD,16 GB,Windows 11 Home,No,33.78 cm (13.3 inch),1920 x 1200 Pixel,0.97 kg,"₹82,555"
981,22,LENOVO,Laptop,Processing & Multitasking,AMD Ryzen 7 Octa Core,512 GB,NVIDIA GeForce RTX 3050,16 GB,Windows 11 Home,No,39.62 cm (15.6 inch),1920 x 1080 Pixels,2.4 Kg,"₹1,04,999"
982,23,DELL,Thin and Light Laptop,Processing & Multitasking,Intel Core i5 11th Gen,512 GB,Intel Integrated Iris Xe,16 GB,Windows 10,No,34.04 cm (13.4 inch),3840 x 2400 Pixels,1.27 Kg,"₹1,37,490"
